Below section imports the required libraries for the implementation. It includes libraries like NumPy, scikit-learn (for KNN, SVM, decision trees, random forests), TensorFlow, and PyTorch for deep learning.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from torch import nn, optim
import torch

Below section generates a synthetic dataset using the Labeled Faces in the Wild (LFW) dataset, which is available in scikit-learn.

In [2]:
from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

Split the dataset into training and testing sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    lfw_people.data, lfw_people.target, test_size=0.25, random_state=42
)

PCA for dimensionality reduction (Eigenfaces)

In [4]:
n_components = 150
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

K-Nearest Neighbors (KNN)

In [5]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train_pca, y_train)
knn_predictions = knn_classifier.predict(X_test_pca)
knn_accuracy = accuracy_score(y_test, knn_predictions)

print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.6583850931677019


Support Vector Machine (SVM)

In [6]:
svm_classifier = SVC()
svm_classifier.fit(X_train_pca, y_train)
svm_predictions = svm_classifier.predict(X_test_pca)
svm_accuracy = accuracy_score(y_test, svm_predictions)

print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.7981366459627329


Decision Trees

In [7]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_pca, y_train)
dt_predictions = dt_classifier.predict(X_test_pca)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Trees Accuracy:", dt_accuracy)

Decision Trees Accuracy: 0.38198757763975155


Random Forests

In [8]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_pca, y_train)
rf_predictions = rf_classifier.predict(X_test_pca)
rf_accuracy = accuracy_score(y_test, rf_predictions)

print("Random Forests Accuracy:", rf_accuracy)

Random Forests Accuracy: 0.5683229813664596


Deep Learning with TensorFlow

In [9]:
model_tf = keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(n_components,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(np.unique(y_train)), activation='softmax')
])

model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_tf.fit(X_train_pca, y_train, epochs=10, batch_size=64, validation_split=0.2)
tf_accuracy = model_tf.evaluate(X_test_pca, y_test, verbose=0)[1]
print("TensorFlow Accuracy:", tf_accuracy)


Epoch 1/10
13/13 [==============================] - 3s 54ms/step - loss: 1.9759 - accuracy: 0.3044 - val_loss: 1.5220 - val_accuracy: 0.4485
Epoch 2/10
13/13 [==============================] - 0s 14ms/step - loss: 1.3561 - accuracy: 0.5220 - val_loss: 1.3172 - val_accuracy: 0.5567
Epoch 3/10
13/13 [==============================] - 0s 14ms/step - loss: 1.0064 - accuracy: 0.6749 - val_loss: 1.1531 - val_accuracy: 0.6392
Epoch 4/10
13/13 [==============================] - 0s 14ms/step - loss: 0.7566 - accuracy: 0.8148 - val_loss: 1.0299 - val_accuracy: 0.6959
Epoch 5/10
13/13 [==============================] - 0s 14ms/step - loss: 0.6241 - accuracy: 0.8355 - val_loss: 0.9450 - val_accuracy: 0.7216
Epoch 6/10
13/13 [==============================] - 0s 14ms/step - loss: 0.4857 - accuracy: 0.8964 - val_loss: 0.8708 - val_accuracy: 0.7371
Epoch 7/10
13/13 [==============================] - 0s 13ms/step - loss: 0.3930 - accuracy: 0.9313 - val_loss: 0.8166 - val_accuracy: 0.7680
Epoch 8/10
13

Deep Learning with PyTorch

In [10]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(n_components, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, len(np.unique(y_train)))

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model_pt = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pt.parameters(), lr=0.001)


Convert numpy arrays to PyTorch tensors

In [11]:
X_train_pca_pt = torch.from_numpy(X_train_pca).float()
y_train_pt = torch.from_numpy(y_train).long()
X_test_pca_pt = torch.from_numpy(X_test_pca).float()
y_test_pt = torch.from_numpy(y_test).long()

Training loop

In [12]:
epochs = 10

for epoch in range(epochs):
    model_pt.train()
    optimizer.zero_grad()
    outputs = model_pt(X_train_pca_pt)
    loss = criterion(outputs, y_train_pt)
    loss.backward()
    optimizer.step()

Testing

In [13]:
model_pt.eval()
with torch.no_grad():
    pt_predictions = torch.argmax(model_pt(X_test_pca_pt), axis=1).numpy()
    pt_accuracy = accuracy_score(y_test, pt_predictions)
    
    print("PyTorch Accuracy:", pt_accuracy)

PyTorch Accuracy: 0.5124223602484472
